In [1]:
import sqlite3
import pandas as pd
import ast
import warnings
warnings.filterwarnings('ignore')

conn = sqlite3.Connection("myhome.db")
cursor = conn.cursor()

In [2]:
df=pd.read_sql_query('SELECT * FROM homes',conn)
df=df.replace('None',None)

In [3]:
df=df.drop_duplicates(subset='URL')
df=df.dropna(subset='ADDRESS')

In [4]:
df=df.reset_index(drop=True)

In [5]:
df.shape

(7262, 8)

In [6]:
df=df[df['ADDRESS'].str.contains('იყიდება')]

In [7]:
city_parts=df['ADDRESS'].apply(lambda x : str(x).split('ბინა ')[-1].split('\n')[0].strip())
df.insert(df.columns.get_loc('ADDRESS')+1,'CityPart',city_parts)

import datetime
dates=df['ADDRESS'].apply(lambda x : ' '.join(str(x).split('\n')[1].split(' ')[:-1]))
georgian_month_names = {'იან': 'Jan', 'თებ': 'Feb', 'მარ': 'Mar', 'აპრ': 'Apr', 'მაი': 'May', 'ივნ': 'Jun', 'ივლ': 'Jul', 'აგვ': 'Aug', 'სექ': 'Sep', 'ოქტ': 'Oct', 'ნოე': 'Nov', 'დეკ': 'Dec'}
dates=dates.apply(lambda x : str(x).split(' ')[0]+' '+georgian_month_names.get(str(x).split(' ')[-1],'')).str.strip()
df['SCRAPE_DATE']=pd.to_datetime(df['SCRAPE_DATE'])
dates=[
 df['SCRAPE_DATE'].iloc[i].strftime('%d-%m-%Y') if date == "დღეს" 
 else (df['SCRAPE_DATE'].iloc[i] - datetime.timedelta(days=1)).strftime('%d-%m-%Y') if date == "გუშინ" 
 else date 
 for i, date in enumerate(dates)
 ]
dates=pd.Series(dates)
dates=dates.apply(lambda x:str(x)+', 2024' if '2024' not in str(x) else str(x))
dates=dates.apply(pd.to_datetime)
df.insert(df.columns.get_loc('ADDRESS')+2,'ArticlePublishDate',dates)

df=df.drop(columns='ADDRESS')

In [8]:
price_gel=df['PRICE'].apply(lambda x : x.split('\n')[0].replace(',','') if '₾' in str(x) else None).apply(pd.to_numeric)
df.insert(df.columns.get_loc('PRICE')+1,'PRICE (GEL)',price_gel)
df=df.drop(columns='PRICE')

In [9]:
area=df['DETAILS'].apply(lambda x : x.split('საერთო ფართი\n')[-1].split('\n')[0].split(' ')[0]).apply(pd.to_numeric)
rooms=df['DETAILS'].apply(lambda x : x.split('ოთახი\n')[-1].split('\n')[0].replace('+','')).apply(pd.to_numeric)
bedrooms=df['DETAILS'].apply(lambda x : x.split('საძინებელი\n')[-1].split('\n')[0].replace('+','')).apply(pd.to_numeric,errors='coerce')
floor=df['DETAILS'].apply(lambda x : x.split('სართული\n')[-1].split('\n')[0].split('/')[0]).apply(pd.to_numeric,errors='coerce')
max_floor=df['DETAILS'].apply(lambda x : x.split('სართული\n')[-1].split('\n')[0].split('/')[-1]).apply(pd.to_numeric,errors='coerce')
last_floor=(floor==max_floor).astype(int)

df.insert(df.columns.get_loc('DETAILS')+1,'Area',area)
df.insert(df.columns.get_loc('DETAILS')+2,'Rooms',rooms)
df.insert(df.columns.get_loc('DETAILS')+3,'Bedrooms',bedrooms)
df.insert(df.columns.get_loc('DETAILS')+4,'Floor',floor)
df.insert(df.columns.get_loc('DETAILS')+1,'MaxFloor',max_floor)
df.insert(df.columns.get_loc('DETAILS')+5,'LastFloor',last_floor)

df=df.drop(columns='DETAILS')

In [10]:
import ast

df['PARAMETERS']=df['PARAMETERS'].apply(ast.literal_eval)
param_df=pd.json_normalize(df['PARAMETERS'])

In [11]:
param_df_useful_columns=list(param_df.isna().mean()[param_df.isna().mean()<0.3].index)
param_df=param_df[param_df_useful_columns]

In [12]:
param_df['სვ.წერტილები']=param_df['სვ.წერტილები'].str.replace('+','').apply(pd.to_numeric,errors='coerce')
param_df['ბუნ. აირი']=param_df['ბუნ. აირი'].map({'კი':1,'არა':0}).fillna(0)

In [13]:
col_index=df.columns.get_loc('PARAMETERS')
df=pd.concat([df.iloc[:,:col_index],param_df,df.iloc[:,col_index+1:]],axis=1)

In [14]:
furniture_list_series=df['FURNITURE'].apply(
    lambda x: [i.strip() for i in str(x).split('\n') if i.strip() != 'ავეჯი'] if isinstance(x, str) else None
)

all_columns=[]
for x in furniture_list_series:
    if isinstance(x, list):
        for i in x:
            all_columns.append(i)
all_columns=list(set(all_columns))

furniture_df = pd.DataFrame(0, index=range(len(furniture_list_series)), columns=all_columns)

for i, furniture_list in enumerate(furniture_list_series):
    if isinstance(furniture_list, list):
        for value in furniture_list:
            if value in all_columns:
                furniture_df.loc[i, value]=1

In [15]:
df=pd.concat([df,furniture_df],axis=1)

df=df.drop(columns=['FURNITURE'])

In [16]:
df=df.drop(columns=['URL','STREET','პროექტის ტიპი','ArticlePublishDate','SCRAPE_DATE'])

In [17]:
df=df.dropna(subset='PRICE (GEL)')

### Translate into English

In [18]:
from googletrans import Translator
translator=Translator()


def transliterate_georgian(word):
    translit_map = {
        'ა': 'a', 'ბ': 'b', 'გ': 'g', 'დ': 'd', 'ე': 'e', 'ვ': 'v', 
        'ზ': 'z', 'თ': 't', 'ი': 'i', 'კ': 'k', 'ლ': 'l', 'მ': 'm',
        'ნ': 'n', 'ო': 'o', 'პ': 'p', 'ჟ': 'zh', 'რ': 'r', 'ს': 's', 
        'ტ': 't', 'უ': 'u', 'ფ': 'f', 'ქ': 'k', 'ღ': 'gh', 'ყ': 'q',
        'შ': 'sh', 'ჩ': 'ch', 'ც': 'ts', 'ძ': 'dz', 'წ': 'ts', 'ჭ': 'ch',
        'ხ': 'kh', 'ჯ': 'j', 'ჰ': 'h'
    }
    return ''.join(translit_map.get(char, char) for char in word)

for col in ["CityPart"]:
    col_unique=df[col].unique()
    word_dict={}
    for word in col_unique:

        if pd.isnull(word):
            word_dict[word]=word
        else:
            transliterated_word=transliterate_georgian(word)
            transliterated_word=transliterated_word.title()
            word_dict[word]=transliterated_word

    df[col]=df[col].map(word_dict)    

    print(col,end='\r')


In [19]:
df['სტატუსი']=df['სტატუსი'].map({
    'ახალი აშენებული': 'Newly Constructed',
    'მშენებარე': 'Under Construction',
    'ძველი აშენებული': 'Old Constructed'
})

In [20]:
mapping_dict = {
    'ახალი გარემონტებული': 'Newly Renovated',
    'შავი კარკასი': 'Black Frame',
    'მწვანე კარკასი': 'Green Frame',
    'ძველი გარემონტებული': 'Old Renovated',
    'თეთრი კარკასი': 'White Frame',
    'სარემონტო': 'Needs Renovation',
    'მიმდინარე რემონტი': 'Ongoing Renovation',
    'თეთრი პლიუსი': 'White Plus',
}
df['მდგომარეობა'] = df['მდგომარეობა'].map(mapping_dict)


In [21]:
mapping_dict_heating = {
    'ცენტრალური გათბობა': 'Central Heating',
    'გაზის გამაცხელებელი': 'Gas Heater',
    'იატაკის გათბობა': 'Underfloor Heating',
    'ელექტრო გამაცხელებელი': 'Electric Heater',
    'ცენტრალური + იატაკის გათბობა': 'Central + Underfloor Heating',
    'გათბობის გარეშე': 'No Heating',
    'ინდივიდუალური': 'Individual Heating'
}
df['გათბობა'] = df['გათბობა'].map(mapping_dict_heating)


In [23]:
df.columns=[
    'CityPart', 
    'MaxFloor', 
    'Area', 
    'Rooms', 
    'Bedrooms', 
    'LastFloor', 
    'Floor', 
    'PRICE (GEL)', 
    'Status', 
    'Electrical Points', 
    'Condition', 
    'Heating', 
    'Natural Gas', 
    'Bed', 
    'Stove (Gas/Electric)', 
    'Table', 
    'Refrigerator', 
    'Chairs', 
    'Oven', 
    'Air Conditioner', 
    'Washing Machine', 
    'Sofa', 
    'Dishwasher'
]


In [24]:
df.to_csv('Tbilisi Apartment Prices (Detailed).csv',index=False)

In [27]:
df=pd.read_csv('Tbilisi Apartment Prices (Detailed).csv')

In [28]:
df

,CityPart,MaxFloor,Area,Rooms,Bedrooms,LastFloor,Floor,PRICE (GEL),Status,Electrical Points,...,Bed,Stove (Gas/Electric),Table,Refrigerator,Chairs,Oven,Air Conditioner,Washing Machine,Sofa,Dishwasher
0,Krtsanisshi,6.0,730.0,7.0,4.0,1.0,6.0,4619070.0,Newly Constructed,3.0,...,0,0,1,0,0,0,0,0,0,0
1,Didubeshi,30.0,44.0,2.0,1.0,0.0,15.0,149441.0,Under Construction,1.0,...,0,0,0,0,0,0,0,0,0,0
2,Didubeshi,30.0,95.0,3.0,2.0,0.0,24.0,234757.0,Under Construction,1.0,...,0,0,0,0,0,0,0,0,0,0
3,Didubeshi,30.0,95.0,3.0,2.0,0.0,28.0,234757.0,Under Construction,1.0,...,0,0,0,0,0,0,0,0,0,0
4,Didubeshi,30.0,128.0,4.0,3.0,0.0,18.0,347789.0,Under Construction,1.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7253,Did Dighomshi,10.0,90.0,4.0,3.0,0.0,6.0,346252.0,Under Construction,2.0,...,0,0,0,0,0,0,0,0,0,0
7254,Saburtaloze,8.0,80.0,3.0,2.0,0.0,7.0,271570.0,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
7255,Chughuretshi,8.0,253.0,6.0,3.0,0.0,7.0,1072702.0,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
7256,Did Dighomshi,9.0,76.0,2.0,1.0,0.0,4.0,257992.0,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
